In [1]:
!git clone https://github.com/ShanmukhVegi/Image-Classification.git

Cloning into 'Image-Classification'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (116/116), 17.16 MiB | 38.79 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
%cd /content/Image-Classification

/content/Image-Classification


In [3]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import pickle

In [4]:
Categories=['Cars','Ice cream cone','Cricket ball']
print("Type y to give categories or type n to go with classification of Cars,Ice Cream cone and Cricket ball");

while(True):
  check=input()
  if(check=='n' or check=='y'):
    break
  print("Please give a valid input (y/n)")
if(check=='y'):
  print("Enter How Many types of Images do you want to classify")
  n=int(input())
  Categories=[]
  print(f'please enter {n} names')
  for i in range(n):
    name=input()
    Categories.append(name)
  print(f"If not drive Please upload all the {n} category images in google collab with the same names as given in categories")


Type y to give categories or type n to go with classification of Cars,Ice Cream cone and Cricket ball
n


In [5]:
flat_data_arr=[]
target_arr=[]
#please use datadir='/content' if the files are upload on to google collab
#else mount the drive and give path of the parent-folder containing all category images folders.
datadir='/content/Image-Classification'
for i in Categories:
  print(f'loading... category : {i}')
  path=os.path.join(datadir,i)
  for img in os.listdir(path):
    img_array=imread(os.path.join(path,img))
    img_resized=resize(img_array,(150,150,3))
    flat_data_arr.append(img_resized.flatten())
    target_arr.append(Categories.index(i))
  print(f'loaded category:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data)
df['Target']=target
df

loading... category : Cars
loaded category:Cars successfully
loading... category : Ice cream cone
loaded category:Ice cream cone successfully
loading... category : Cricket ball
loaded category:Cricket ball successfully


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,67461,67462,67463,67464,67465,67466,67467,67468,67469,67470,67471,67472,67473,67474,67475,67476,67477,67478,67479,67480,67481,67482,67483,67484,67485,67486,67487,67488,67489,67490,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.604291,0.584683,0.600369,0.607843,0.588235,0.603922,0.611765,0.592157,0.607843,0.611765,0.592157,0.607843,0.608261,0.588654,0.604340,0.611765,0.592157,0.607843,0.617798,0.598190,0.613876,0.619608,0.600000,0.615686,0.619608,0.600000,0.615686,0.623529,0.603922,0.619608,0.623111,0.603503,0.619190,0.623529,0.603922,0.619608,0.627451,0.607843,0.623529,0.627451,...,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0.976471,0
1,0.874510,0.827529,0.796078,0.874510,0.823529,0.792157,0.878523,0.831464,0.800000,0.867203,0.823529,0.788235,0.874585,0.831373,0.796153,0.878431,0.831373,0.800000,0.870588,0.828447,0.797074,0.866667,0.823502,0.795987,0.870649,0.831373,0.796139,0.878431,0.839216,0.803922,0.886275,0.847059,0.811765,0.886275,0.839216,0.805775,0.882353,0.831373,0.800000,0.882353,...,0.333425,0.364797,0.470680,0.327608,0.352899,0.458781,0.309712,0.341047,0.442954,0.303416,0.337163,0.439067,0.313725,0.341176,0.446967,0.326758,0.353060,0.460092,0.320573,0.345190,0.450052,0.317739,0.349111,0.451072,0.325598,0.361673,0.462853,0.317830,0.364261,0.459007,0.317647,0.349020,0.450889,0.317647,0.352941,0.454816,0.325582,0.360784,0.462837,0
2,0.843137,0.843137,0.835294,0.850980,0.850980,0.843137,0.858824,0.858824,0.847240,0.866013,0.866486,0.854902,0.870588,0.870588,0.867209,0.878431,0.878431,0.874510,0.882353,0.882353,0.881699,0.888235,0.888235,0.886275,0.894118,0.894118,0.890196,0.897858,0.897858,0.894118,0.901961,0.901961,0.901961,0.905882,0.905882,0.909804,0.909804,0.909804,0.909804,0.913725,...,0.219135,0.130316,0.070115,0.240551,0.141608,0.074941,0.292699,0.163830,0.065248,0.284758,0.157739,0.030719,0.199388,0.116562,0.038562,0.208928,0.124072,0.054902,0.200000,0.113725,0.049242,0.203490,0.113725,0.035294,0.214059,0.122987,0.038131,0.221359,0.125309,0.038131,0.204394,0.117647,0.042052,0.201418,0.119608,0.050980,0.198734,0.125490,0.078431,0
3,0.105882,0.094118,0.121569,0.105882,0.094118,0.121569,0.105882,0.098039,0.121569,0.106275,0.098039,0.118039,0.076078,0.068235,0.086667,0.031373,0.023529,0.028627,0.031373,0.023529,0.027451,0.027451,0.023529,0.027451,0.027451,0.023529,0.027451,0.027451,0.023529,0.027451,0.027451,0.023529,0.027451,0.027451,0.023529,0.027451,0.027451,0.023529,0.027451,0.027451,...,0.182353,0.152941,0.152941,0.172549,0.149020,0.149020,0.176471,0.152941,0.152941,0.176471,0.152941,0.160784,0.184314,0.160784,0.168627,0.182941,0.159412,0.167255,0.178824,0.155294,0.163137,0.167882,0.144353,0.152196,0.161961,0.138431,0.146275,0.152941,0.129412,0.137255,0.146471,0.125490,0.133333,0.137255,0.121569,0.125490,0.133333,0.117647,0.121569,0
4,0.492105,0.601908,0.637203,0.499948,0.601908,0.637203,0.507791,0.601908,0.641124,0.507791,0.601908,0.641124,0.507791,0.601908,0.641124,0.511712,0.605830,0.645046,0.511712,0.605830,0.645046,0.511712,0.605830,0.645046,0.510758,0.604876,0.644092,0.501856,0.595974,0.635190,0.496928,0.591046,0.631268,0.484157,0.578275,0.623425,0.476314,0.570431,0.615582,0.478327,...,0.489190,0.391150,0.268627,0.497033,0.395072,0.274510,0.505882,0.400954,0.282353,0.507791,0.403922,0.278431,0.507791,0.403922,0.278431,0.508797,0.404928,0.276471,0.509804,0.407843,0.278431,0.513725,0.407843,0.282353,0.513725,0.411765,0.282353,0.521569,0.415686,0.286275,0.525490,0.415686,0.288288,0.519660,0.415686,0.290196,0.517647,0.411765,0.290196,0
...,...,...,...,...,...,...,...,...,...,...,...,.

In [6]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('Splitted Successfully')

Splitted Successfully


In [7]:
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
svc=svm.SVC(probability=True)
print("The training of the model is started, please wait for while as it may take few minutes to complete")
model=GridSearchCV(svc,param_grid)
model.fit(x_train,y_train)
print('The Model is trained well with the given images')
model.best_params_

The training of the model is started, please wait for while as it may take few minutes to complete
The Model is trained well with the given images


{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

In [8]:
y_pred=model.predict(x_test)
print("The predicted Data is :")
y_pred

The predicted Data is :


array([0, 0, 2, 2, 0, 1, 1, 2, 2, 1, 2, 2, 0, 1, 2, 0, 2, 0, 1, 1])

In [9]:
print("The actual data is:")
np.array(y_test)

The actual data is:


array([0, 0, 2, 2, 0, 1, 1, 2, 0, 1, 2, 2, 0, 1, 2, 0, 0, 0, 1, 1])

In [10]:
#classification_report(y_pred,y_test)
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")
#confusion_matrix(y_pred,y_test)

The model is 90.0% accurate


In [ ]:
pickle.dump(model,open('img_model.p','wb'))
print("Pickle is dumped successfully")

In [ ]:
#print(os.path.abspath(os.getcwd()))
model=pickle.load(open('img_model.p','rb'))

url=input('Enter URL of Image')
img=imread(url)
plt.imshow(img)
plt.show()
img_resize=resize(img,(150,150,3))
l=[img_resize.flatten()]
probability=model.predict_proba(l)
for ind,val in enumerate(Categories):
  print(f'{val} = {probability[0][ind]*100}%')
print("The predicted image is : "+Categories[model.predict(l)[0]])
print(f'Is the image a {Categories[model.predict(l)[0]]} ?(y/n)')
while(True):
  b=input()
  if(b=="y" or b=="n"):
    break
  print("please enter either y or n")

if(b=='n'):
  print("What is the image?")
  for i in range(len(Categories)):
    print(f"Enter {i} for {Categories[i]}")
  k=int(input())
  while(k<0 or k>=len(Categories)):
    print(f"Please enter a valid number between 0-{len(Categories)-1}")
    k=int(input())
  print("Please wait for a while for the model to learn from this image :)")
  flat_arr=flat_data_arr.copy()
  tar_arr=target_arr.copy()
  tar_arr.append(k)
  flat_arr.extend(l)
  tar_arr=np.array(tar_arr)
  flat_df=np.array(flat_arr)
  df1=pd.DataFrame(flat_df)
  df1['Target']=tar_arr
  model1=GridSearchCV(svc,param_grid)
  x1=df1.iloc[:,:-1]
  y1=df1.iloc[:,-1]
  x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.20,random_state=77,stratify=y1)
  d={}
  for i in model.best_params_:
    d[i]=[model.best_params_[i]]
  model1=GridSearchCV(svc,d)
  model1.fit(x_train1,y_train1)
  y_pred1=model.predict(x_test1)
  print(f"The model is now {accuracy_score(y_pred1,y_test1)*100}% accurate")
  pickle.dump(model1,open('img_model.p','wb'))
print("Thank you for your feedback")
